In [14]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import seaborn
seaborn.set_context('talk')

from sqlalchemy import create_engine, text

import plotly

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2024*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

#### Ingesta de datos desde el servidor de la UNC

In [23]:
melb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')


print(melb_df.columns)

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


In [16]:
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

C:\Users\Fran\AppData\Local\Temp\ipykernel_1712\750777605.py:7: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pd.read_csv(


## Ejercicio 1 SQL:

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  



#### 1. Creación de base de datos SQLite con SQLalchemy
>**Consigna:** 
>
>Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
>https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

In [17]:
# Crea base de datos local 
motor = create_engine("sqlite:///data/melb_data.db")

#### 2. Construir tabla en base al .csv ingestado
Se utiliza el comando de pandas **DataFrame.to_sql** para generar una consulta **SQL** del tipo **DDL** (Data Definition Language o Lenguaje de Definición de Datos) para crear una tabla compatible con los datos obtenidos.

> **Consigna:**
>
> Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus > precios por codigo postal en otra.
> 


In [19]:
# Creación de tabla llamada users con los datos de melb_data.csv
with motor.begin() as conexion:
    melb_df.to_sql(name="melb_data", con=conexion, if_exists='replace')
    airbnb_df.to_sql(name="airbnb", con=conexion, if_exists='replace')
    


#### 3. Consultas SQL

> **Consigna:**
>
> Implementar consultas en SQL que respondan con la siguiente información:
>   - cantidad de registros totales por ciudad.
> 
>   - cantidad de registros totales por barrio y ciudad.

**Consultas a ejecutar:**

In [35]:
################################ Cantidad de registros totales por ciudad ################################
sql_totales_ciudad = """SELECT CouncilArea, COUNT(1) AS Cantidad 
FROM melb_data
GROUP BY CouncilArea
ORDER BY CouncilArea ASC;
""" 
########################## Cantidad de registros totales por barrio y ciudad #############################
sql_totales_ciudad_barrio = """SELECT CouncilArea, Suburb, COUNT(1) AS Cantidad 
FROM melb_data
GROUP BY CouncilArea, Suburb
ORDER BY CouncilArea, Suburb ASC;
""" 

**Ejecución -> Cantidad de registros totales por ciudad:**

In [33]:
with motor.begin() as conexion:
    rs = conexion.execute(text(sql_totales_ciudad))

for r in rs:
    print(r)

(None, 1369)
('Banyule', 594)
('Bayside', 489)
('Boroondara', 1160)
('Brimbank', 424)
('Cardinia', 8)
('Casey', 38)
('Darebin', 934)
('Frankston', 53)
('Glen Eira', 848)
('Greater Dandenong', 52)
('Hobsons Bay', 434)
('Hume', 164)
('Kingston', 207)
('Knox', 80)
('Macedon Ranges', 7)
('Manningham', 311)
('Maribyrnong', 692)
('Maroondah', 80)
('Melbourne', 470)
('Melton', 66)
('Monash', 333)
('Moonee Valley', 997)
('Moorabool', 1)
('Moreland', 1163)
('Nillumbik', 36)
('Port Phillip', 628)
('Stonnington', 719)
('Unavailable', 1)
('Whitehorse', 304)
('Whittlesea', 167)
('Wyndham', 86)
('Yarra', 647)
('Yarra Ranges', 18)


**Ejecución -> Cantidad de registros totales por ciudad y barrio:**

In [37]:
with motor.begin() as conexion:
    rs = conexion.execute(text(sql_totales_ciudad_barrio))

for r in rs:
    print(r)

(None, 'Abbotsford', 1)
(None, 'Aberfeldie', 4)
(None, 'Airport West', 6)
(None, 'Albanvale', 2)
(None, 'Albert Park', 5)
(None, 'Albion', 3)
(None, 'Alphington', 2)
(None, 'Altona', 5)
(None, 'Altona Meadows', 2)
(None, 'Altona North', 7)
(None, 'Ardeer', 1)
(None, 'Armadale', 9)
(None, 'Ascot Vale', 5)
(None, 'Ashburton', 7)
(None, 'Ashwood', 1)
(None, 'Aspendale', 3)
(None, 'Aspendale Gardens', 2)
(None, 'Attwood', 1)
(None, 'Avondale Heights', 1)
(None, 'Balaclava', 2)
(None, 'Balwyn', 6)
(None, 'Balwyn North', 20)
(None, 'Bayswater', 2)
(None, 'Bayswater North', 3)
(None, 'Beaumaris', 7)
(None, 'Bellfield', 1)
(None, 'Bentleigh', 8)
(None, 'Bentleigh East', 15)
(None, 'Berwick', 2)
(None, 'Black Rock', 4)
(None, 'Blackburn', 8)
(None, 'Blackburn North', 7)
(None, 'Blackburn South', 4)
(None, 'Bonbeach', 3)
(None, 'Boronia', 2)
(None, 'Box Hill', 5)
(None, 'Braybrook', 2)
(None, 'Briar Hill', 3)
(None, 'Brighton', 10)
(None, 'Brighton East', 10)
(None, 'Broadmeadows', 6)
(None, 'Br

#### 4. Consulta SQL: JOINS 

> **Consigna:**
>
> Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase. 

**Consulta SQL:**

In [39]:
sql_join = """SELECT * FROM """

## Ejercicio 2 - Pandas:

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.


2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico.
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)